# Optuna

[optuna](https://github.com/pfnet/optuna) はパラメータチューニングを自動的にやってくれるやつです

In [1]:
import optuna
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2.) ** 2

In [3]:
study = optuna.create_study()

In [4]:
study.optimize(objective, n_trials=100)

[I 2019-02-26 18:42:45,301] Finished a trial resulted in value: 19.106628488709823. Current best value is 19.106628488709823 with parameters: {'x': -2.371112957669914}.
[I 2019-02-26 18:42:45,314] Finished a trial resulted in value: 1.5568410908112496. Current best value is 1.5568410908112496 with parameters: {'x': 0.7522656168834452}.
[I 2019-02-26 18:42:45,327] Finished a trial resulted in value: 0.6326149671904089. Current best value is 0.6326149671904089 with parameters: {'x': 1.2046290380015066}.
[I 2019-02-26 18:42:45,334] Finished a trial resulted in value: 130.30513448911924. Current best value is 0.6326149671904089 with parameters: {'x': 1.2046290380015066}.
[I 2019-02-26 18:42:45,339] Finished a trial resulted in value: 28.58876040585548. Current best value is 0.6326149671904089 with parameters: {'x': 1.2046290380015066}.
[I 2019-02-26 18:42:45,345] Finished a trial resulted in value: 55.51372484108158. Current best value is 0.6326149671904089 with parameters: {'x': 1.2046290

[I 2019-02-26 18:42:45,683] Finished a trial resulted in value: 2.7890130229825996. Current best value is 0.0008598422349755203 with parameters: {'x': 1.9706769333974852}.
[I 2019-02-26 18:42:45,691] Finished a trial resulted in value: 9.283185404046337. Current best value is 0.0008598422349755203 with parameters: {'x': 1.9706769333974852}.
[I 2019-02-26 18:42:45,702] Finished a trial resulted in value: 137.04236644896903. Current best value is 0.0008598422349755203 with parameters: {'x': 1.9706769333974852}.
[I 2019-02-26 18:42:45,711] Finished a trial resulted in value: 40.565277245655594. Current best value is 0.0008598422349755203 with parameters: {'x': 1.9706769333974852}.
[I 2019-02-26 18:42:45,726] Finished a trial resulted in value: 3.2266271603262275. Current best value is 0.0008598422349755203 with parameters: {'x': 1.9706769333974852}.
[I 2019-02-26 18:42:45,736] Finished a trial resulted in value: 24.79003974862925. Current best value is 0.0008598422349755203 with parameter

[I 2019-02-26 18:42:46,253] Finished a trial resulted in value: 4.21589512804267e-06. Current best value is 4.21589512804267e-06 with parameters: {'x': 2.002053264505134}.
[I 2019-02-26 18:42:46,271] Finished a trial resulted in value: 2.184235587605056. Current best value is 4.21589512804267e-06 with parameters: {'x': 2.002053264505134}.
[I 2019-02-26 18:42:46,290] Finished a trial resulted in value: 108.86801218858982. Current best value is 4.21589512804267e-06 with parameters: {'x': 2.002053264505134}.
[I 2019-02-26 18:42:46,304] Finished a trial resulted in value: 61.434785051937084. Current best value is 4.21589512804267e-06 with parameters: {'x': 2.002053264505134}.
[I 2019-02-26 18:42:46,318] Finished a trial resulted in value: 13.573328493319114. Current best value is 4.21589512804267e-06 with parameters: {'x': 2.002053264505134}.


In [5]:
study.best_params

{'x': 2.002053264505134}

In [6]:
df = study.trials_dataframe()

In [7]:
df.columns

MultiIndex(levels=[['datetime_complete', 'datetime_start', 'params', 'state', 'trial_id', 'value'], ['', 'x']],
           codes=[[4, 3, 5, 1, 0, 2], [0, 0, 0, 0, 0, 1]])

## trial object

trial が持つ関数 `suggest_categorical`, `suggest_uniform`, `suggest_loguniform`, `suggest_discrete_uniform`

## てすと

scikit learn で実験する

In [8]:
from sklearn.datasets import load_boston

In [9]:
data = load_boston()

In [10]:
data.keys()

dict_keys(['filename', 'DESCR', 'target', 'feature_names', 'data'])

In [11]:
x, y = data['data'], data['target']

In [12]:
x.shape

(506, 13)

まずは SVR の初期パラメータでチューニングしてみる

In [13]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

In [14]:
score = 0
n_train = 5
for idx_train, idx_valid in KFold(n_train, random_state=19).split(x, y):
    x_train, y_train = x[idx_train], y[idx_train]
    x_valid, y_valid = x[idx_valid], y[idx_valid]
    scalar = StandardScaler()
    x_train = scalar.fit_transform(x_train)
    x_valid = scalar.transform(x_valid)
    clf = SVR()
    clf.fit(x_train, y_train)    
    pred = clf.predict(x_valid)
    score += mean_squared_error(y_valid, pred) ** .5

In [15]:
print('default SVR: {}'.format(score / n_train))

default SVR: 6.56728896457391


In [16]:
def svm_objective(trial):
    x, y = load_boston(return_X_y=True)
    
    use_scaling = True
    params = {
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'linear', 'sigmoid', 'poly']),
        'C': trial.suggest_loguniform('C', 1e-3, 1000.),
        'epsilon': trial.suggest_loguniform('epsilon', 1e-3, 10.)
    }
    
    if params.get('kernel') == 'poly':
        params['degree'] = trial.suggest_int('degree', 1, 7)
    
    score = 0
    n_train = 5
    for idx_train, idx_valid in KFold(n_train, random_state=19).split(x, y):
        x_train, y_train = x[idx_train], y[idx_train]
        x_valid, y_valid = x[idx_valid], y[idx_valid]
        if use_scaling:
            scalar = StandardScaler()
            x_train = scalar.fit_transform(x_train)
            x_valid = scalar.transform(x_valid)
        clf = SVR(**params)
        clf.fit(x_train, y_train)    
        pred = clf.predict(x_valid)
        score += mean_squared_error(y_valid, pred) ** .5
    
    return score / n_train

In [17]:
study = optuna.create_study()

In [18]:
study.optimize(svm_objective, n_trials=50)

[I 2019-02-26 18:42:48,269] Finished a trial resulted in value: 8.39403867281028. Current best value is 8.39403867281028 with parameters: {'epsilon': 0.010691030690599835, 'kernel': 'sigmoid', 'C': 0.02089644139225051}.
[I 2019-02-26 18:42:48,341] Finished a trial resulted in value: 7.153636268426899. Current best value is 7.153636268426899 with parameters: {'epsilon': 0.009006623748755863, 'kernel': 'sigmoid', 'C': 1.170804814920216}.
[I 2019-02-26 18:42:48,387] Finished a trial resulted in value: 5.771970283131902. Current best value is 5.771970283131902 with parameters: {'epsilon': 0.0014505923809048295, 'kernel': 'linear', 'C': 0.024590131848390567}.
[I 2019-02-26 18:42:48,446] Finished a trial resulted in value: 7.909512239325636. Current best value is 5.771970283131902 with parameters: {'epsilon': 0.0014505923809048295, 'kernel': 'linear', 'C': 0.024590131848390567}.
[I 2019-02-26 18:42:48,505] Finished a trial resulted in value: 8.736700882213182. Current best value is 5.7719702

[I 2019-02-26 18:42:56,815] Finished a trial resulted in value: 5.500905729277157. Current best value is 5.406667555388433 with parameters: {'epsilon': 0.004653897454822331, 'kernel': 'linear', 'C': 0.260345726522815}.
[I 2019-02-26 18:42:56,887] Finished a trial resulted in value: 7.26997518888856. Current best value is 5.406667555388433 with parameters: {'epsilon': 0.004653897454822331, 'kernel': 'linear', 'C': 0.260345726522815}.
[I 2019-02-26 18:42:56,943] Finished a trial resulted in value: 9.11448679477228. Current best value is 5.406667555388433 with parameters: {'epsilon': 0.004653897454822331, 'kernel': 'linear', 'C': 0.260345726522815}.
[I 2019-02-26 18:42:57,026] Finished a trial resulted in value: 338.15145045938823. Current best value is 5.406667555388433 with parameters: {'epsilon': 0.004653897454822331, 'kernel': 'linear', 'C': 0.260345726522815}.
[I 2019-02-26 18:42:57,106] Finished a trial resulted in value: 5.714290667264847. Current best value is 5.406667555388433 wi

In [19]:
study.trials_dataframe().sort_values('value').head()

trial_id                state     value             datetime_start  \
                                                                        
44       44  TrialState.COMPLETE  5.391509 2019-02-26 18:43:02.066097   
26       26  TrialState.COMPLETE  5.406668 2019-02-26 18:42:55.901319   
7         7  TrialState.COMPLETE  5.445513 2019-02-26 18:42:48.686714   
48       48  TrialState.COMPLETE  5.481215 2019-02-26 18:43:04.434350   
29       29  TrialState.COMPLETE  5.487400 2019-02-26 18:42:56.115402   

            datetime_complete     params                           
                                       C degree   epsilon  kernel  
44 2019-02-26 18:43:02.210751  35.914381    NaN  0.089322     rbf  
26 2019-02-26 18:42:55.974450   0.260346    NaN  0.004654  linear  
7  2019-02-26 18:42:48.782667   9.483774    NaN  0.014734     rbf  
48 2019-02-26 18:43:04.513742   0.751952    NaN  0.024946  linear  
29 2019-02-26 18:42:56.171247   0.067121    NaN  0.004336  linear

In [20]:
import lightgbm as lgbm

In [21]:
score = 0
n_train = 5
for idx_train, idx_valid in KFold(n_train, random_state=19).split(x, y):
    x_train, y_train = x[idx_train], y[idx_train]
    x_valid, y_valid = x[idx_valid], y[idx_valid]
    clf = lgbm.LGBMRegressor()
    clf.fit(x_train, y_train)    
    pred = clf.predict(x_valid)
    score += mean_squared_error(y_valid, pred) ** .5

In [22]:
score / n_train

4.571009019613884

In [23]:
def lgbm_objective(trial):
    x, y = load_boston(return_X_y=True)
    params = {
        'n_estimators': 200,
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e2),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e2),
        'subsample_bytree': trial.suggest_uniform('subsample_bytree', .5, 1.),
        'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
    }
    n_train = 5
    score = .0
    for idx_train, idx_valid in KFold(n_train, random_state=19).split(x, y):
        x_train, y_train = x[idx_train], y[idx_train]
        x_valid, y_valid = x[idx_valid], y[idx_valid]
        clf = lgbm.LGBMRegressor(**params)
        clf.fit(x_train, y_train, verbose=False)    
        pred = clf.predict(x_valid)
        score += mean_squared_error(y_valid, pred) ** .5
    return score / n_train

In [24]:
study_lgbm = optuna.create_study()

In [25]:
study_lgbm.optimize(lgbm_objective, n_trials=50)

[I 2019-02-26 18:43:05,485] Finished a trial resulted in value: 4.830657622287058. Current best value is 4.830657622287058 with parameters: {'reg_lambda': 27.077133750709667, 'subsample_bytree': 0.6353327757639637, 'reg_alpha': 0.2821206716700904, 'boosting': 'goss'}.
[I 2019-02-26 18:43:05,681] Finished a trial resulted in value: 5.737959293038861e+36. Current best value is 4.830657622287058 with parameters: {'reg_lambda': 27.077133750709667, 'subsample_bytree': 0.6353327757639637, 'reg_alpha': 0.2821206716700904, 'boosting': 'goss'}.
[I 2019-02-26 18:43:05,911] Finished a trial resulted in value: 1.2973376910618233e+30. Current best value is 4.830657622287058 with parameters: {'reg_lambda': 27.077133750709667, 'subsample_bytree': 0.6353327757639637, 'reg_alpha': 0.2821206716700904, 'boosting': 'goss'}.
[I 2019-02-26 18:43:06,434] Finished a trial resulted in value: 4.4983267531119155. Current best value is 4.4983267531119155 with parameters: {'reg_lambda': 0.01696374257625363, 'subsa

[I 2019-02-26 18:43:15,854] Finished a trial resulted in value: 8.54982143942257e+37. Current best value is 4.444804555848971 with parameters: {'reg_lambda': 0.0019260298200966953, 'subsample_bytree': 0.9416758638472609, 'reg_alpha': 8.148515352610913, 'boosting': 'dart'}.
[I 2019-02-26 18:43:16,173] Finished a trial resulted in value: 4.634390114397578. Current best value is 4.444804555848971 with parameters: {'reg_lambda': 0.0019260298200966953, 'subsample_bytree': 0.9416758638472609, 'reg_alpha': 8.148515352610913, 'boosting': 'dart'}.
[I 2019-02-26 18:43:16,318] Finished a trial resulted in value: 1.0421308584325018e+37. Current best value is 4.444804555848971 with parameters: {'reg_lambda': 0.0019260298200966953, 'subsample_bytree': 0.9416758638472609, 'reg_alpha': 8.148515352610913, 'boosting': 'dart'}.
[I 2019-02-26 18:43:16,840] Finished a trial resulted in value: 4.521145210621123. Current best value is 4.444804555848971 with parameters: {'reg_lambda': 0.0019260298200966953, '

In [26]:
study_lgbm.trials_dataframe().sort_values('value').head()

trial_id                state     value             datetime_start  \
                                                                        
24       24  TrialState.COMPLETE  4.444805 2019-02-26 18:43:14.011253   
23       23  TrialState.COMPLETE  4.480039 2019-02-26 18:43:13.583326   
39       39  TrialState.COMPLETE  4.491570 2019-02-26 18:43:18.438442   
17       17  TrialState.COMPLETE  4.491992 2019-02-26 18:43:11.329845   
47       47  TrialState.COMPLETE  4.495418 2019-02-26 18:43:20.897896   

            datetime_complete   params                                        
                              boosting reg_alpha reg_lambda subsample_bytree  
24 2019-02-26 18:43:14.294450     dart  8.148515   0.001926         0.941676  
23 2019-02-26 18:43:14.006584     dart  0.132266   0.001021         0.871140  
39 2019-02-26 18:43:18.866774     dart  0.003517   0.229400         0.999888  
17 2019-02-26 18:43:11.766112     dart  0.439125   0.014237         0.554362  
47 2019-02-26 18:43:21.277138     dart  0.001114   0.007949         0.854646